# Introduction to FastAPI

**remark** - you cannot run the servers from this notebook. You must use a terminal then cd into this directory and then run the website. You can then access the website using for example the `requests` library.

## Chapter 1 - FastAPI Basics

### Section 1.1 - Why FastAPI

#### First application
Let's run the FastAPI server for the first time! You can't run the FastAPI server directly with "Run this file" - see the instructions for how to run and stop the server from the terminal.

In [6]:
%%writefile main.py
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
def root():
    return {"message": "Hello World"}

Overwriting main.py


run the app by executuring the following command in terminal from this directory

In [10]:
!curl http://localhost:8000

{"message":"Hello World"}

### Section 1.2 - GET operations

#### Hello world

Let's build your first GET endpoint! You can't run the FastAPI server directly with "Run this file" - see the instructions for how to run and stop the server from the terminal.

In [17]:
%%writefile main.py
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
def root():
    return {"message": "Hello World"}

@app.get("/hello")
def hello(name: str = "Alan"):
    return {"message": f"Hello {name}"}

Overwriting main.py


In [12]:
!curl http://localhost:8000

{"message":"Hello World"}

In [13]:
!curl http://localhost:8000/hello

{"message":"Hello Alan"}

In [18]:
!curl -H 'Content-Type: application/jsons' http://localhost:8000/hello

{"message":"Hello Alan"}

In [19]:
import requests

response = requests.get(
    "http://localhost:8000",
    headers={"Content-Type": "application/json"},
    params={"name": "Steve"}
)

print(response.status_code)
print(response.json())

200
{'message': 'Hello World'}


### Section 1.3 - POST operations

#### Pydantic model

You've been asked to create an API endpoint that manages items in inventory. To get started, create a Pydantic model for Items that has attributes name, quantity, and expiration.

In [25]:
# Import date
from datetime import date

# Import BaseModel
from pydantic import BaseModel

# Define model Item
class Item(BaseModel):
    name: str
    quantity: int = 0
    expiration: date = None

#### POST operation in action
You've been asked to create an API endpoint that accepts a `name` parameter and returns a message saying "We have name". To accomplish this, create a Pydantic model for Item and root endpoint (/) that serves HTTP POST operations. The endpoint should accept the `Item` model as input and respond with a message including `Item.name`.

You can't run the FastAPI server directly with "Run this file" - see the instructions for how to run the server and test your code from the terminal.

In [28]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel

# Define model Item
class Item(BaseModel):
    name: str

app = FastAPI()


@app.post("/")
def root(item: Item):
    name = item.name
    return {"message": f"We have {name}"}

Overwriting main.py


In [36]:
!curl -X POST -H 'Content-Type: application/json' -d '{"name": "bananas"}' http://localhost:8000

{"message":"We have bananas"}

In [35]:
import requests

response = requests.post(
    "http://localhost:8000",
    headers={"Content-Type": "application/json"},
    json={"name": "Steve"}
)

print(response.status_code)
print(response.json())

200
{'message': 'We have Steve'}


## Chapter 2 - FastAPI Advanced topics

### Section 2.1 PUT and DELETE operations

#### PUT operation in action
You've been asked to create a PUT endpoint `/items` that accepts parameters `name` and `description` and updates the `description` based on the `name` in a key-value store called `items`.

You can't run the FastAPI server directly with "Run this file" - see the instructions for how to run the server and test your code from the terminal.

In [44]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel

# Define model Item
class Item(BaseModel):
    name: str
    description: str

# Define items at application start
items = {"bananas": "Yellow fruit."}

app = FastAPI()


@app.put("/items")
def update_item(item: Item):
    name = item.name
    # Update the description
    items[name] = item.description
    return item

Overwriting main.py


In [45]:
!curl -X PUT -H 'Content-Type: application/json' -d '{"name": "bananas", "description": "Delicious!"}' http://localhost:8000/items

{"name":"bananas","description":"Delicious!"}

In [48]:
data = {"name": "bananas", "description": "Delicious!"}
headers={"Content-Type": "application/json"}
response = requests.put("http://localhost:8000/items", json=data, headers=headers)
response.json()

{'name': 'bananas', 'description': 'Delicious!'}

#### DELETE operation in action

You've been asked to create a DELETE endpoint that accepts parameter name and deletes the item called `name` from a key store called `items`.

You can't run the FastAPI server directly with "Run this file" - see the instructions for how to run the server and test your code from the terminal.

In [69]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel

# Define model Item
class Item(BaseModel):
    name: str

# Define items at application start
items = {"apples", "oranges", "bananas"}

app = FastAPI()


@app.delete("/items")
def delete_item(item: Item):
    name = item.name
    # Delete the item
    items.remove(name)
    return {}

Overwriting main.py


In [70]:
!curl -X DELETE -H 'Content-Type: application/json' -d '{"name": "bananas"}' http://localhost:8000/items

{}

In [71]:
url = "http://localhost:8000/items"
data = {"name": "apples"}
headers = {'Content-Type': 'application/json'}
response = requests.delete(url, json=data, headers=headers)
response.json()

{}

### Section 2.2 - Handling errors

#### Handling a client error

You've been asked to create a DELETE endpoint that accepts parameter `name` and deletes the item called name from a key store called `items`. If the item is not found, the endpoint should return an appropriate status code and detailed message.

You can't run the FastAPI server directly with "Run this file" - see the instructions for how to run the server and test your code from the terminal.

In [77]:
%%writefile main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# Define model Item
class Item(BaseModel):
    name: str

# Define items at application startup
items = {"apples", "oranges"}

app = FastAPI()


@app.delete("/items")
def delete_item(item: Item):
    name = item.name
    if name in items:
        items.remove(name)
    else:
        # Raise HTTPException with status code for "not found"
        raise HTTPException(status_code=404, detail="Item not found.")
    return {}

Overwriting main.py


In [79]:
!curl -X DELETE -H 'Content-Type: application/json' -d '{"name": "bananas"}'  http://localhost:8000/items

{"detail":"Item not found."}

In [82]:
url = "http://localhost:8000/items"
data = {"name": "peach"}
headers = {'Content-Type': 'application/json'}
response = requests.delete(url, json=data, headers=headers)
response.json()

{'detail': 'Item not found.'}

### Section 2.3 - Using async for concurrent work

#### Asynchronous DELETE operation
You've been asked to create an API endpoint that deletes items managed by your API. To accomplish this, create an endpoint `/items` that serves HTTP DELETE operations. Make the endpoint asynchronous, so that your application can continue to serve requests while maintaining any long-running deletion tasks.

We can't run the FastAPI server directly with "Run this file" - see the instructions for how to run the server and test your code from the terminal.

In [89]:
%%writefile main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# Define model Item
class Item(BaseModel):
    name: str

app = FastAPI()

items = {"rock", "paper", "scissors"}


@app.delete("/items")
# Make asynchronous
async def root(item: Item):
    name = item.name
    # Check if name is in items
    if name not in items:
        # Return the status code for not found
        raise HTTPException(status_code=404, detail="Item not found.")
    items.remove(name)
    return {"message": "Item deleted"}

Overwriting main.py


In [90]:
url = "http://localhost:8000/items"
data = {"name": "rock"}
headers = {'Content-Type': 'application/json'}
response = requests.delete(url, json=data, headers=headers)
print(response.json())

data = {"name": "roll"}
headers = {'Content-Type': 'application/json'}
response = requests.delete(url, json=data, headers=headers)
print(response.json())


{'message': 'Item deleted'}
{'detail': 'Item not found.'}


## Chapter 3 - FastAPI automated testing

In [93]:
from fastapi.testclient import TestClient
#from .main import app

client = TestClient(app)

#### System test
You've built your FastAPI application and added unit tests to verify code functionality. Writing a system test for an API endpoint will ensure that the endpoint works on the running application.

We can't run the FastAPI server directly with "Run this file" - see the instructions for how to run the server and test your code from the terminal.

In [96]:
%%writefile main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional

# define model Item
class Item(BaseModel):
    name: str
    quantity: Optional[int] = 0

app = FastAPI()

items = {"scissors": Item(name="scissors", quantity=100)}


@app.get("/items")
def read(name: str):
    print(name)
    if name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    return items[name]

Overwriting main.py


In [102]:
import requests

url = "http://localhost:8000/items"
params = {"name": "scissors"}  # this goes into the query string

response = requests.get(url, params=params)

print(response.status_code)
print(response.json())

200
{'name': 'scissors', 'quantity': 100}


In [103]:
%%writefile system_test.py
# Import TestClient
from fastapi.testclient import TestClient
from main import app

# Create test client with application context
client = TestClient(app)

def test_main():
    response = client.get("/items?name=scissors")
    assert response.status_code == 200
    assert response.json() == {"name": "scissors",
                               "quantity": 100}

Writing system_test.py


In [104]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/sacha/fastapi
configfile: pyproject.toml
plugins: anyio-4.9.0, cov-6.1.1, hydra-core-1.3.2, langsmith-0.3.33
collected 1 item                                                               

system_test.py .                                                         [100%]

============================== 1 passed in 0.49s ===============================
